In [1]:
import sys 
sys.path.append('/home/w4756677/garment/AIpparel-Code/src')
from transformers import AutoProcessor, LlavaConfig
import os 

from data.datasets.gcd_dataset import GarmentCodeData
from data.datasets.panel_configs import StandardizeConfig, StatsConfig
from data.garment_tokenizers.garment_tokenizer_for_regression import GarmentTokenizerForRegression
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/w4756677/miniconda3/envs/llama3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = GarmentCodeData(
    root_dir="/miele/timur/garmentcodedata",
    editing_dir="/mnt/gs/sci-garment/garmentcodedata_editing",
    caption_dir="/mnt/gs/sci-garment/long-caption-processed",
    editing_flip_prob=0.5,
    sampling_rate=[0, 0, 0, 0, 1],
    split_file="/home/w4756677/garment/AIpparel-Code/assets/garmentcodedata_datasplit.json",
    datalist_file="/home/w4756677/garment/AIpparel-Code/assets/garmentcodedata_list.txt",
    panel_classification="/home/w4756677/garment/AIpparel-Code/assets/panel_classes_garmentcodedata.json",
    body_type="default_body",
    split="train"
)
val_dataset = GarmentCodeData(
    root_dir="/miele/timur/garmentcodedata",
    editing_dir="/mnt/gs/sci-garment/garmentcodedata_editing",
    caption_dir="/mnt/gs/sci-garment/long-caption-processed",
    editing_flip_prob=0.5,
    sampling_rate=[0.5, 0.5,0,0,0],
    split_file="/home/w4756677/garment/AIpparel-Code/assets/garmentcodedata_datasplit.json",
    datalist_file="/home/w4756677/garment/AIpparel-Code/assets/garmentcodedata_list.txt",
    panel_classification="/home/w4756677/garment/AIpparel-Code/assets/panel_classes_garmentcodedata.json",
    body_type="default_body",
    split="val"
)

gt_stats = StandardizeConfig(
    rotations=StatsConfig(shift=[0, 0, 0, 0], scale=[1, 1, 1, 1]),
    translations=StatsConfig(shift=[-1.25378371e-02,  1.13507532e+02,  2.63046369e+00], scale=[26.06867645, 32.42920198, 22.29905009]),
    vertices=StatsConfig(shift=[8.44428116, 16.84081321], scale=[24.4920733,  26.60402835])
)

garment_tokenizer = GarmentTokenizerForRegression(
    standardize=gt_stats,
    random_tag=True,
    num_tags=108,
    include_template_name=False
)

PanelClasses::INFO::Loading panel classes from file:  /home/w4756677/garment/AIpparel-Code/assets/panel_classes_garmentcodedata.json
The panel classes in this dataset are : ['left_btorso', 'left_ftorso', 'right_ftorso', 'right_btorso', 'skirt_front', 'skirt_back', 'wb_back', 'wb_front', 'left_sleeve_b', 'left_sleeve_f', 'right_sleeve_f', 'right_sleeve_b', 'sl_left_cuff_skirt_f', 'sl_left_cuff_skirt_b', 'sl_right_cuff_skirt_b', 'sl_right_cuff_skirt_f', 'sl_left_cuff_b', 'sl_left_cuff_f', 'sl_right_cuff_f', 'sl_right_cuff_b', 'pant_f_l', 'pant_b_l', 'pant_f_r', 'pant_b_r', 'left_collar_front', 'left_collar_back', 'right_collar_front', 'right_collar_back', 'ins_skirt_front_0', 'ins_skirt_back_0', 'ins_skirt_back_1', 'ins_skirt_front_1', 'skirt_panel_2', 'skirt_panel_0', 'skirt_panel_1', 'skirt_panel_3', 'skirt_panel_4', 'skirt_panel_5', 'ins_skirt_back_2', 'ins_skirt_front_2', 'skirt_panel_6', 'skirt_front_0', 'skirt_back_0', 'left_hood', 'right_hood', 'skirt_panel_7', 'skirt_panel_8', 'p

In [3]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("meta-llama/Llama-3.2-11B-Vision-Instruct")
# processor.tokenizer.add_tokens("<pad>", special_tokens=True)
# processor.tokenizer.pad_token = "<pad>"
processor.tokenizer.pad_token = "<|finetune_right_pad_id|>"
all_new_tokens = garment_tokenizer.get_all_token_names()
num_added_tokens = processor.tokenizer.add_tokens(all_new_tokens, special_tokens=True)
token_name2_idx_dict = {}
for token in all_new_tokens:
    token_idx = processor.tokenizer(token, add_special_tokens=False).input_ids[0]
    token_name2_idx_dict[token] = token_idx
    
garment_tokenizer.set_token_indices(token_name2_idx_dict)

In [5]:
from data.collate_fns import collate_fn
print(len(processor.tokenizer))
data = train_dataset[0]
dialog = data[2]
chat = processor.apply_chat_template(dialog, tokenize=False)
data_dict = collate_fn(
    [train_dataset[0], train_dataset[1], train_dataset[2], train_dataset[3]], 
    processor=processor, 
    garment_tokenizer=garment_tokenizer,
    model_version="meta-llama/Llama-3.2-11B-Vision-Instruct")

128379


In [ ]:
print(data_dict["input_ids"].shape, data_dict["labels"].shape)
from tqdm import tqdm
for i in tqdm(range(len(train_dataset))):
    data_dict = collate_fn(
    [train_dataset[i]], 
    processor=processor, 
    garment_tokenizer=garment_tokenizer,
    model_version="meta-llama/Llama-3.2-11B-Vision-Instruct")
    if 128265 in data_dict["pattern_params"].keys():
        if data_dict["pattern_params"][128265].shape[-1] != 2:
            print(i, data_dict["pattern_params"][128265].shape)
# print(processor.tokenizer.decode([128256, 128265]))
# print(data_dict["labels"].max(), data_dict["labels"][data_dict["labels"] != -100].min())
# for k in data_dict["pattern_params"].keys():
#     print(k)
#     print(data_dict["pattern_params"][k][data_dict["pattern_params_mask"][k]].shape)

torch.Size([1, 279]) torch.Size([1, 279])


In [4]:
system_prompt="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."
sample_dict = {}
sample_dict["user"] = "Question: What do respiration and combustion give out\nChoices:\nA. Oxygen\nB. Carbon dioxide\nC. Nitrogen\nD. Heat\nAnswer with the letter."
sample_dict["assistant"] = "Answer: B"
dialog = [
    {"role":"system","content":[{"type": "text", "text": system_prompt}]},
    {"role":"user","content":[{"type": "image"}, {"type": "text", "text": sample_dict["user"].strip()}]},
    {"role":"assistant","content":[{"type": "text", "text": sample_dict["assistant"].strip()}]}
]
sample_dict["user"] = "Question: In the given food web, which are the organism that only eaten roadrunner?\nChoices:\nA. dingo, jack rabbit\nB. coyote, bobcat\nC. dingo, bobcat\nD. snake, jack rabbit\nAnswer with the letter."
sample_dict["assistant"] = "Answer: B"
dialog += [
    {"role":"user","content":[{"type": "text", "text": sample_dict["user"].strip()}]},
    {"role":"assistant","content":[{"type": "text", "text": sample_dict["assistant"].strip()}]}
]

In [5]:
from transformers import AutoTokenizer,LlavaNextProcessor
# dialog = [
#     {"role":"user","content":[
#         {"type": "image"}, 
#         {"type": "text", "text": sample_dict["user"].strip()}
#         ]},
#     {"role":"assistant","content":[
#         {"type": "text", "text": sample_dict["assistant"].strip()}
#         ]}
# ]
# dialog = [
#     {
#         "role": "user",
#         "content": [
#             {"type": "image"},
#             {"type": "text", "text": "What is shown in this image?"},
#         ],
#     },
# ]
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
text_prompt = processor.apply_chat_template(dialog, tokenize=False, add_generation_prompt=False)
print(text_prompt)


<<SYS>>
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.
<</SYS>>

[INST] <image>
Question: What do respiration and combustion give out
Choices:
A. Oxygen
B. Carbon dioxide
C. Nitrogen
D. Heat
Answer with the letter. [/INST] Answer: B<\s> [INST] Question: In the given food web, which are the organism that only eaten roadrunner?
Choices:
A. dingo, jack rabbit
B. coyote, bobcat
C. dingo, bobcat
D. snake, jack rabbit
Answer with the letter. [/INST] Answer: B<\s> 


In [7]:
text_prompt = processor.apply_chat_template(dialog, tokenize=False, add_generation_prompt=False)
batch = processor(images=[image], text=[text_prompt],padding = True, return_tensors="pt")
import sys 
import numpy as np 
np.set_printoptions(threshold=sys.maxsize)
print(batch.input_ids.numpy())
print((batch.input_ids == 32000).sum())

[[    1  2087 18741  4060    13 28741 10706  1444   264 13903  2188   304
    396 18278 10895 13892 28723   415 13892  5212 10865 28725 10537 28725
    304 27057 11194   298   272  2188 28742 28713  4224 28723    13 28789
    700 18741  4060    13    13 28792 16289 28793 28705 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000 32000
  32000 32000 32000 32000 32000 32000 

In [29]:
print(processor.decode([733, 28748, 16289, 28793, 26307, 28747, 365, 16910, 28713, 28767]))
print(processor.decode([16910, 28713, 28767]))
print(processor.decode([28792, 16289, 28793, 28705]))
print(processor.decode([733, 28748, 16289, 28793]))
print(processor.decode([2087, 18741, 4060]))
print(processor.decode([700, 18741,  4060]))
print(processor.tokenizer.pad_token_id)

[/INST] Answer: B<\s>
<\s>
[INST] 
[/INST]
<<SYS>>
</SYS>>
32001


In [37]:
from datasets import load_dataset

raw_datasets = load_dataset("HuggingFaceH4/llava-instruct-mix-vsft")
train_dataset =raw_datasets["train"]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/259155 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13640 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [40]:
text_prompt = processor.apply_chat_template([train_dataset[0]], tokenize=False, add_generation_prompt=False)


UndefinedError: 'dict object' has no attribute 'role'